In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
path = '/content/drive/MyDrive/2o Cuatri/TFG/scripts/'

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import keras.backend as K
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, TimeDistributed, LSTM, \
    Bidirectional, Reshape, \
    Embedding, Dropout, Flatten, BatchNormalization, \
    RNN, concatenate, Activation
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

In [3]:
rn.seed(123)
np.random.seed(99)
tf.random.set_seed(1234)
# other parameters
MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 300
nb_words = 3438

In [4]:
# load feature and labels
feat = np.load(path+'npy/pAA-lld-100.npy')

feat = feat.reshape(10039, 100, 34)

vad = np.load(path+'npy/emotions.npy')
# print(vad[:5].T[0])

# remove outlier, < 1, > 5
vad = np.where(vad == 5.5, 5.0, vad)
vad = np.where(vad == 0.5, 1.0, vad)

print(f'feature shape: {feat.shape} validation shape: {vad.shape}')

feature shape: (10039, 100, 34) validation shape: (10039, 3)


In [5]:
# text feature
x_train_text = np.load(f'{path}npy/text_seq_{MAX_SEQUENCE_LENGTH}.npy')

# maxlen is 554, but we will trim it each sentence to mean+stdev = 58+51 = 109
# also, will reshape to use 109 timesteps of 1 words in lstm
# x_train_text = np.delete(x_train_text, np.s_[-109:], 1)
# x_train_text = x_train_text.reshape(10039, 109, 1)

g_word_embedding_matrix = np.load(path+'npy/embeddings.npy')

print(f'text shape: {x_train_text.shape} embeddings shape: {g_word_embedding_matrix.shape}')
x_train_text[1234]

text shape: (10039, 554) embeddings shape: (3438, 300)


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [6]:
scaler = StandardScaler()
scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
scaled_feat = scaler.transform(feat.reshape(
    feat.shape[0]*feat.shape[1], feat.shape[2]))
scaled_feat = scaled_feat.reshape(
    feat.shape[0], feat.shape[1], feat.shape[2])

print(f'features: {feat[0][0][:]}')
feat = scaled_feat
print(f'Scaled features: {feat[0][0][:]}')

features: [ 6.26566416e-02  1.62956941e-03  3.08895291e+00  1.69652208e-01
  2.13637464e-01  5.27793982e-01  0.00000000e+00  9.00000000e-02
 -2.78627205e+01  2.45111414e+00  1.51317008e-01  4.56337041e-01
  4.12160915e-02  5.54149736e-01 -1.95606531e-01 -5.89442197e-02
  1.16821299e-01 -1.56289678e-01 -3.54407482e-01 -2.12553956e-01
  1.70777537e-02  4.28046482e-03  1.03589055e-03  7.43697928e-02
  4.09309022e-03  2.68172445e-02  4.07779226e-04  1.13720459e-01
  6.52234816e-05  8.02601465e-03  1.61389691e-02  2.12453263e-02
  6.67289518e-04  3.40113745e-02]
Scaled features: [ 0.12515368 -0.22970145 -0.15445826  0.63791088  1.09633723 -0.07639218
 -0.23457344  0.03474504  0.96302889  1.10517717  0.34971886  1.13925872
  0.35447286  1.6734508  -0.71672234 -0.13190179  0.68303697 -0.52122585
 -1.3007472  -0.90332181  0.28289273 -0.0519008  -0.03335487  1.26627094
 -0.00920418  0.68147918 -0.29339772 -0.77231995 -0.20918975  0.49595634
  0.66614776  0.26321299 -0.14042694 -0.92527321]


In [7]:
scaler = MinMaxScaler(feature_range=(-1, 1))
# .reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
scaler = scaler.fit(vad)
# .reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
scaled_vad = scaler.transform(vad)

print(f'Validation set: {vad[0][:]}')
vad = scaled_vad
print(f'Scaled validation set: {vad[0][:]}')

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold = K.squeeze(gold, axis=-1)
    pred = K.squeeze(pred, axis=-1)
    gold_mean = K.mean(gold, axis=-1, keepdims=True)
    pred_mean = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc = K.constant(2.) * covariance / (gold_var + pred_var +
                                         K.square(gold_mean - pred_mean))
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss = K.constant(1.) - ccc(gold, pred)
    return ccc_loss



Validation set: [3.  2.5 2.5]
Scaled validation set: [ 0.   -0.25 -0.25]


In [10]:
import matplotlib.pyplot as plt
def plot_training(history):
  # summarize history for ccc
  plt.figure(figsize=(10,10))
  plt.plot(history.history['v_ccc'], color='green', label='v_ccc')
  plt.plot(history.history['a_ccc'], color='blue', label='a_ccc')
  plt.plot(history.history['d_ccc'], color='red', label= 'd_ccc')
  plt.title(f'Model Train CCC')
  plt.ylabel('ccc')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()
  
  plt.figure(figsize=(10,10))
  plt.plot(history.history['val_v_ccc'], color='green', label= 'val_v_ccc')
  plt.plot(history.history['val_a_ccc'], color='blue', label= 'val_a_ccc')
  plt.plot(history.history['val_d_ccc'], color='red', label= 'val_d_ccc')
  plt.title(f'Model Validation CCC')
  plt.ylabel('ccc')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

  # summarize history for loss
  plt.figure(figsize=(10,10))
  plt.plot(history.history['v_loss'], color='green', label='v_loss')
  plt.plot(history.history['a_loss'], color='blue', label='a_loss')
  plt.plot(history.history['d_loss'], color='red', label= 'd_loss')
  plt.title(f'Model Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

  plt.figure(figsize=(10,10))
  plt.plot(history.history['val_v_loss'], color='green', label= 'val_v_loss')
  plt.plot(history.history['val_a_loss'], color='blue', label= 'val_a_loss')
  plt.plot(history.history['val_d_loss'], color='red', label= 'val_d_loss')
  plt.title(f'Model Validation Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

In [8]:
def create_model(alpha, beta, gamma):
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(50, return_sequences=True)(net_speech)
    net_speech = LSTM(25, return_sequences=True)(net_speech)
    net_speech = LSTM(25, return_sequences=True)(net_speech)
    net_speech = Flatten()(net_speech)
    model_speech = Dropout(0.3)(net_speech)

    # text network
    input_text = Input(shape=(x_train_text.shape[1]), ) #, x_train_text.shape[2]))
    net_text = Embedding(nb_words,
                         EMBEDDING_DIM,
                         weights=[g_word_embedding_matrix],
                         trainable=True)(input_text)
    net_text = Reshape((x_train_text.shape[1], 300))(net_text)
    net_text = LSTM(256, return_sequences=True)(net_text)
    net_text = LSTM(128, return_sequences=True)(net_text)
    net_text = LSTM(128, return_sequences=False)(net_text)
    net_text = Dense(64)(net_text)
    model_text = Dropout(0.3)(net_text)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    # model_combined = [Dense(1, name=name)(model_combined)
    #                   for name in target_names]
    model_valence = Dense(1, name='v')(model_combined)
    model_arousal = Dense(1, name='a')(model_combined)
    model_dominance = Dense(1, name='d')(model_combined)

    model_combined = [model_valence, model_arousal, model_dominance]

    model = Model([input_speech, input_text], model_combined)
    model.compile(loss=ccc_loss,
                  loss_weights={'v': alpha, 'a': beta, 'd': gamma},
                  optimizer='adam', metrics=[ccc])
    return model



In [9]:
results = []
histories = []
for i in range(10):
  print(f'Iteration {i}\n')

  model = create_model(0.7, 0.2, 0.1)
  plot_model(model, show_shapes=True)

  # 7869 first data of session 5 (for LOSO)
  earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=15, # change: patience from 10 to 15
                            restore_best_weights=True)
  hist = model.fit([feat[:7869], x_train_text[:7869]],
                  {"v": vad[:7869].T[0], "a": vad[:7869].T[1],
                      "d": vad[:7869].T[2]},
                  batch_size=256,  # best:8
                  validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                  callbacks=[earlystop])
  metrik = model.evaluate(
      [feat[7869:], x_train_text[7869:]],
      {"v": vad[7869:].T[0], "a": vad[7869:].T[1], "d": vad[7869:].T[2]})
  # print("CCC: ", metrik[-3:])  # np.mean(metrik[-3:]))
  # print("CCC_mean: ", np.mean(metrik[-3:]))
  results.append(metrik[-3:])
  histories.append(hist)
  # plot_training(hist)

# visualize trainig
ccc_list = [np.mean(r) for r in results]
v_list = [r[0] for r in results]
a_list = [r[1] for r in results]
d_list = [r[2] for r in results]

print(f'ccc_mean: {np.mean(ccc_list)} +- {np.std(ccc_list)} \n \
        valence_mean: {np.mean(v_list)} +- {np.std(v_list)} \n \
        arousal_mean: {np.mean(a_list)} +- {np.std(a_list)} \n \
        dominance_mean: {np.mean(d_list)} +- {np.std(d_list)} \n' )

print(f'Plot of best model: Model {np.argmax(ccc_list)} \t ccc_mean = {ccc_list[np.argmax(ccc_list)]}\t {results[np.argmax(ccc_list)]}')
plot_training(histories[np.argmax(ccc_list)])

Epoch 1/50
25/25 [==============================] - 51s 484ms/step - loss: 0.8730 - v_loss: 0.9193 - a_loss: 0.7358 - d_loss: 0.8229 - v_ccc: 0.0807 - a_ccc: 0.2642 - d_ccc: 0.1771 - val_loss: 0.6498 - val_v_loss: 0.7069 - val_a_loss: 0.4820 - val_d_loss: 0.5861 - val_v_ccc: 0.2931 - val_a_ccc: 0.5180 - val_d_ccc: 0.4139
Epoch 2/50
25/25 [==============================] - 10s 402ms/step - loss: 0.5923 - v_loss: 0.6219 - a_loss: 0.4894 - d_loss: 0.5903 - v_ccc: 0.3781 - a_ccc: 0.5106 - d_ccc: 0.4097 - val_loss: 0.5726 - val_v_loss: 0.5943 - val_a_loss: 0.4997 - val_d_loss: 0.5662 - val_v_ccc: 0.4057 - val_a_ccc: 0.5003 - val_d_ccc: 0.4338
Epoch 3/50
25/25 [==============================] - 10s 400ms/step - loss: 0.4759 - v_loss: 0.4691 - a_loss: 0.4595 - d_loss: 0.5561 - v_ccc: 0.5309 - a_ccc: 0.5405 - d_ccc: 0.4439 - val_loss: 0.4845 - val_v_loss: 0.4810 - val_a_loss: 0.4433 - val_d_loss: 0.5910 - val_v_ccc: 0.5190 - val_a_ccc: 0.5567 - val_d_ccc: 0.4090
Epoch 4/50
25/25 [=============

NameError: ignored